<a href="https://colab.research.google.com/github/czarinadamienne/GRP6_CUDAPROJECT/blob/main/GRP6_CUDAPROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing **Histogram Counting** and using **Shared memory concept** in CUDA

### Check CUDA

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

## (1) C Implementation

In [ ]:
%%writefile C_histcount.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

//histogram counting
void histcount(size_t n, int32_t* histbins, int32_t *vec){
  int index = 0;
  for (int i = 0; i < n; i++){
    index = vec[i] % 10;
    histbins[index]++;
  }
}

//reset histbins
void clearhistbins(size_t n, int32_t* histbins){
    for (int i = 0; i < n; i++){
        histbins[i] = 0;
    }
}

int main(int argc, char** argv){
  const size_t size = 10; //change to 1 << 28
  const size_t arr_bytes = size * sizeof(int32_t);

  const size_t repeat = 30;

  //dynamically allocate
  int32_t *vec;
  vec = (int32_t*)malloc(arr_bytes);
  int32_t histbins[10] = {0};

  //time test
  clock_t start, end;

  //initialize arrays with index value
  for (int i = 0; i < size; i++){
    vec[i] = (int32_t) i;
  }

  //avoid cache miss
  histcount(size, histbins, vec);
  clearhistbins(size, histbins);

  //timer
  double elapse, time_taken;
  elapse = 0.0f;

  for(int i = 0; i < repeat; i++){
    start = clock();
    histcount(size, histbins, vec);
    end = clock();
    time_taken = ((double)(end-start))*1E3/CLOCKS_PER_SEC;
    elapse += time_taken;

    if (i < repeat - 1){
        clearhistbins(size, histbins);
    }
  }

  printf("Historgram Bins: \n");
  for (int i = 0; i < 10; i++){
    printf("Historgram Bin #%d: %d\n", i + 1, histbins[i]);
  }

  printf("\n\nC function:\n");
  printf("Average execution time: %f milliseconds\n", elapse/repeat);
  printf("Number of runs: %lu\n", repeat);
  printf("Array size: %lu", size);

  int ind = 0;
  int32_t histcheck[10] = {0};
  for (int i = 0; i < size; i++){
    ind = vec[i] % 10;
    histcheck[ind]++;
  }

  printf("\n\nVector: ");
  for (int i = 0; i < size; i++){
    printf("%d", vec[i]);
  }

  //error checker
  size_t error = 0;
  for (int i = 0; i < size; i++){
    if (histbins[i] != histcheck[i]){
      error++;
    }
  }

  printf("\n\nNumber of errors in C program: %lu\n\n", error);

  //free memory
  free(vec);

  return 0;
}

In [ ]:
%%shell
gcc C_histcount.c -o C_histcount
./C_histcount

## (2) CUDA Implementation

In [ ]:
%%writefile CUDA_histcount.cu

#include <stdio.h>
#include <stdlib.h>

//CUDA conv kernel
__global__
void convolution(size_t n, float *out, float *in){
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

  for (int i = index; i < n - 2; i += stride){
    out[i] = (in[i] + in[i+1] + in[i+2])/3.0f;
  }
}

int main(){
  const size_t size = 1 << 28;
  const size_t arr_bytes = size * sizeof(float);

  const size_t repeat = 10;

  //cuda allocate cpu and gpu memory
  float *in, *out;
  cudaMallocManaged(&in, arr_bytes);
  cudaMallocManaged(&out, arr_bytes);

  //get gpu in
  int device = -1;
  cudaGetDevice(&device);

  //mem advise
  cudaMemAdvise(in, arr_bytes, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(in, arr_bytes, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);


  //prefetch to create CPU page memory
  cudaMemPrefetchAsync(in, arr_bytes, cudaCpuDeviceId, NULL);

  //prefetch to create GPU page memory
  cudaMemPrefetchAsync(out, arr_bytes, device, NULL);

  //initialize arrays with index value
  for (int i = 0; i < size; i++){
    in[i] = (float) i;
  }

  //prefetching CPU-GPU
  cudaMemPrefetchAsync(in, arr_bytes, device, NULL);

  //cuda kernel
  size_t threads = 1024;
  size_t blocks = (size + threads-1) / threads;

  for(int i = 0; i < repeat; i++){
    convolution<<<blocks, threads>>> (size, out, in);
  }

  printf("CUDA kernel:\n");
  printf("Number of blocks: %lu\n", blocks);
  printf("Number of threads: %lu\n", threads);
  printf("Number of runs: %lu\n", repeat);
  printf("Array size: %lu", size);

  cudaDeviceSynchronize(); //wait GPU to finish

  //prefetch from gpu-cpu
  cudaMemPrefetchAsync(out, arr_bytes, cudaCpuDeviceId, NULL);
  cudaMemPrefetchAsync(in, arr_bytes, cudaCpuDeviceId, NULL);

  //error checker
  size_t error = 0;
  for (int i = 0; i < size - 2; i++){
    if (out[i] != (in[i] + in[i+1] + in[i+2])/3.0f){
      error++;
    }
  }
  printf("\n\nNumber of errors in CUDA program: %zu\n", error);

  //free memory
  cudaFree(in);
  cudaFree(out);

  return 0;
}

In [ ]:
%%shell
nvcc -o CUDA_histcount CUDA_histcount.cu
nvprof ./CUDA_histcount.cu